In [4]:
import numpy as np
import galois

This code is provided as supplementary material of the lecture Channel Coding - Graph Based Codes (CC-GBC)

This code illustrates

MAP/ML decoding of a random parity-check code on the BEC

In [5]:
# Code generation
# size of the matrix
m = 12
n = 24

# erasure probablity of BEC
epsilon = 0.4

In [6]:
while(1):
    # construct parity-check matrix
    H = np.random.randint(0, 2, (m,n))

    # for encoding, split matrix into two parts
    # We have a systematic code x = (u p), and we split the parity-check matrix
    # into two parts H = (H_u H_p)
    # The condition H*x^T = 0 reads then H_u*u^T + H_p*p^T = 0
    # which we can solve for p^T = inv(H_p)*H_u*u^T
    # This means that H_p must be invertible, which we check first
    
    Hu = H[:, :m]
    Hp = H[:, m:]

    GF2 = galois.GF(2)
    Hp_gf = GF2(Hp)
    
    # abort construction if H_p is of full rank
    if np.linalg.matrix_rank(Hp_gf) == m:
        break

In [7]:
# Encoding and Decoding using the given code
# encode
k = n-m

# random information word
u = np.random.randint(0, 2, k)

GF2 = galois.GF(2)
Hp_gf = GF2(Hp)
Hu_gf = GF2(Hu)
u_gf = GF2(u)

# parity bits
p = np.linalg.inv(Hp_gf) @ Hu_gf @ u_gf
p = np.array(p, dtype=int)

# codeword
x = np.concatenate([u, p])

# verify if codeword
if not np.all((H @ x) % 2 == 0):
    print("Not a codeword")

In [8]:
# transmission over BEC (-1 marks erasure)
y = x.copy()
mask = np.random.rand(n) < epsilon
y[mask] = -1

# erasure index set
E = np.where(y == -1)[0]
Ebar = np.setdiff1d(np.arange(n), E)

# get sub-matrices
HE = H[:, E]
HEbar = H[:, Ebar]

# calculate s
s = (HEbar @ y[Ebar]) % 2

In [9]:
# can we solve the system of equations?
GF2 = galois.GF(2)
HE_gf = GF2(HE)
r = np.linalg.matrix_rank(HE_gf)

if r == len(E):  # full rank? 
    # if full rank, solve directly using Gaussian elimination
    HEs_gf = GF2(np.column_stack((HE,s)))
    result = HEs_gf.row_reduce()  # Gaußsche Elimination
    xE = result[0:HE.shape[1],-1]

    
    # fill missing positions in y
    y[E] = xE
    
    print("Decoding successful: Transmitted and decoded word:\n")
    print("x = ",x)
    print("y = ",y)

else:
    print("H_E is not of full rank  ... trying to generate the set of all compatible solutions. This will take some time for large n\n")

    numE = len(E)
    xEposs = np.array([[(i >> (numE-1-j)) & 1 for j in range(numE)] for i in range(2**numE)])
    
    compat = []
    for row in xEposs:
        if np.all((HE @ row + s) % 2 == 0):
            compat.append(row)
    
    # all compatible solutions
    X_compat = np.array(compat)
    
    # as there are more than one compatible solution, carry out bitwise MAP
    # decoding
    # find positions that are equal in the set of compatible codewords
    idx_bitwise = np.all(X_compat == X_compat[0], axis=0)
    
    # replace erased positions with recovered bits
    xh_MAP = y.copy()
    xh_MAP[E[idx_bitwise]] = X_compat[0,idx_bitwise]
    
    print(f"Transmitted:\n{x}")
    print(f"Received:\n{y}")
    print(f"Received:\n{y}")
    print(f"Bitwise MAP decoding result:\n{xh_MAP}");   

print("----------------------------------------------------------\n")

Decoding successful: Transmitted and decoded word:

x =  [1 0 1 0 1 0 0 0 0 1 1 0 1 1 0 1 0 1 0 0 0 0 0 0]
y =  [1 0 1 0 1 0 0 0 0 1 1 0 1 1 0 1 0 1 0 0 0 0 0 0]
----------------------------------------------------------

